In [1]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt

from imutils.video import VideoStream
from imutils.video import FPS
import imutils

import math
import time

In [2]:
resW = 1920
resH = 1080
resOutH = 512

cap = cv2.VideoCapture(2)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, resW)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, resH)

cascade = cv2.CascadeClassifier('./haarcascades/barcode_0.2_2000_10.xml')
scale = 1.4
neighbors = 10
objSize = (20, 20)

font = cv2.FONT_HERSHEY_TRIPLEX
textString = ''
colorFPS = (255,255,0)
colorRect = (0, 255, 0)

capture = 0
trigger = 1

In [3]:
while(True):
    tick1 = cv2.getTickCount()
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    xmin = 9999
    xmax = 0
    ymin = 9999
    ymax = 0
    
    codes = cascade.detectMultiScale(gray, scaleFactor=scale, minNeighbors=neighbors, minSize=objSize)
    
    if trigger:
        codesc = codes
        trigger = 0

    for (x,y,w,h) in codes:
        if x < xmin:
            xmin = x
        if (x+w) > xmax:
            xmax = x+w
        if y < ymin:
            ymin = y
        if (y+h) > ymax:
            ymax = y+h
      
    for (x,y,w,h) in codesc:
        if x < xmin:
            xmin = x
        if (x+w) > xmax:
            xmax = x+w
        if y < ymin:
            ymin = y
        if (y+h) > ymax:
            ymax = y+h
    
    img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), colorRect, 3)
    codesc = codes
    
    
    imgOut = img.copy()
    tick2 = cv2.getTickCount()
    tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())
    imgOut = cv2.copyMakeBorder(imgOut, 0, 80, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    textString = str(tick) + " ms"
    cv2.putText(imgOut, textString, (10, resH+30), font, 1, colorFPS, 2, cv2.LINE_AA)
    imgOut = imutils.resize(imgOut, height = resOutH)
    cv2.imshow('output',imgOut)
    
    if capture:
        cv2.imshow('capture', img)
        fileName = "../imageCapture/" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
        cv2.imwrite(fileName, img, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        capture = 0
    
    getKey = cv2.waitKey(10) & 0xFF
    if getKey is ord('c'):
        capture = 1
    elif getKey is ord('q'):
        break

In [4]:
cap.release()
cv2.destroyAllWindows()